In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
k = 64
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [9]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [10]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [11]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [12]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.174827575683594 | KNN Loss: 6.228297710418701 | BCE Loss: 1.9465301036834717
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.177190780639648 | KNN Loss: 6.22810173034668 | BCE Loss: 1.9490892887115479
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.218077659606934 | KNN Loss: 6.228068828582764 | BCE Loss: 1.99000883102417
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.136442184448242 | KNN Loss: 6.2280378341674805 | BCE Loss: 1.90840482711792
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.170071601867676 | KNN Loss: 6.2273850440979 | BCE Loss: 1.9426862001419067
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.154052734375 | KNN Loss: 6.227167129516602 | BCE Loss: 1.9268858432769775
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.193419456481934 | KNN Loss: 6.226790428161621 | BCE Loss: 1.9666287899017334
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.15809440612793 | KNN Loss: 6.226618766784668 | BCE Loss: 1.93147611618042


Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.972324371337891 | KNN Loss: 4.83497428894043 | BCE Loss: 1.1373498439788818
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.987344741821289 | KNN Loss: 4.841643810272217 | BCE Loss: 1.1457008123397827
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.96442174911499 | KNN Loss: 4.795790195465088 | BCE Loss: 1.1686314344406128
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.8169708251953125 | KNN Loss: 4.7224884033203125 | BCE Loss: 1.094482421875
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.812456130981445 | KNN Loss: 4.7007646560668945 | BCE Loss: 1.1116917133331299
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.810124397277832 | KNN Loss: 4.6933417320251465 | BCE Loss: 1.116782546043396
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.793524265289307 | KNN Loss: 4.673505783081055 | BCE Loss: 1.1200186014175415
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.723931789398193 | KNN Loss: 4.6097893714904785 | BCE Loss: 

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 5.5076117515563965 | KNN Loss: 4.457967281341553 | BCE Loss: 1.0496445894241333
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 5.538027763366699 | KNN Loss: 4.485864639282227 | BCE Loss: 1.0521628856658936
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 5.51624870300293 | KNN Loss: 4.441437244415283 | BCE Loss: 1.0748112201690674
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 5.4546966552734375 | KNN Loss: 4.411798000335693 | BCE Loss: 1.042898416519165
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 5.493555068969727 | KNN Loss: 4.437297821044922 | BCE Loss: 1.0562574863433838
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 5.537965774536133 | KNN Loss: 4.481687545776367 | BCE Loss: 1.0562784671783447
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 5.524158000946045 | KNN Loss: 4.479379177093506 | BCE Loss: 1.044778823852539
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 5.468611717224121 | KNN Loss: 4.432379722595215 | BCE Loss

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 5.4424638748168945 | KNN Loss: 4.4166083335876465 | BCE Loss: 1.0258557796478271
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 5.4416279792785645 | KNN Loss: 4.402205467224121 | BCE Loss: 1.0394225120544434
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 5.496085166931152 | KNN Loss: 4.437416076660156 | BCE Loss: 1.058669090270996
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 5.45173454284668 | KNN Loss: 4.408928394317627 | BCE Loss: 1.0428060293197632
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 5.485454082489014 | KNN Loss: 4.417438983917236 | BCE Loss: 1.068015217781067
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 5.485142230987549 | KNN Loss: 4.42959451675415 | BCE Loss: 1.055547833442688
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 5.494961738586426 | KNN Loss: 4.429432392120361 | BCE Loss: 1.0655295848846436
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 5.482602119445801 | KNN Loss: 4.441196918487549 | BCE Loss:

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 5.439749717712402 | KNN Loss: 4.4092912673950195 | BCE Loss: 1.0304585695266724
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 5.3913493156433105 | KNN Loss: 4.368643283843994 | BCE Loss: 1.0227059125900269
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 5.417226791381836 | KNN Loss: 4.386479377746582 | BCE Loss: 1.0307471752166748
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 5.456398963928223 | KNN Loss: 4.393693447113037 | BCE Loss: 1.062705397605896
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 5.4906840324401855 | KNN Loss: 4.430553436279297 | BCE Loss: 1.0601304769515991
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 5.418916702270508 | KNN Loss: 4.407166957855225 | BCE Loss: 1.0117496252059937
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 5.456377983093262 | KNN Loss: 4.4151201248168945 | BCE Loss: 1.0412578582763672
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 5.4453582763671875 | KNN Loss: 4.38686990737915 | BCE 

...

Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 5.415876388549805 | KNN Loss: 4.390794277191162 | BCE Loss: 1.025081992149353
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 5.352718353271484 | KNN Loss: 4.345103740692139 | BCE Loss: 1.0076146125793457
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 5.429523944854736 | KNN Loss: 4.387258529663086 | BCE Loss: 1.0422654151916504
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 5.395442962646484 | KNN Loss: 4.361802577972412 | BCE Loss: 1.0336403846740723
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 5.387103080749512 | KNN Loss: 4.352215766906738 | BCE Loss: 1.034887433052063
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 5.407593727111816 | KNN Loss: 4.380495548248291 | BCE Loss: 1.0270979404449463
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 5.409254550933838 | KNN Loss: 4.3655805587768555 | BCE Loss: 1.043674111366272
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 5.382393836975098 | KNN Loss: 4.349435806274414 | BC

Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 5.3846659660339355 | KNN Loss: 4.376435279846191 | BCE Loss: 1.0082306861877441
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 5.380387306213379 | KNN Loss: 4.3619232177734375 | BCE Loss: 1.0184638500213623
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 5.428438186645508 | KNN Loss: 4.376776218414307 | BCE Loss: 1.0516619682312012
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 5.414963722229004 | KNN Loss: 4.393837928771973 | BCE Loss: 1.0211255550384521
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 5.447108268737793 | KNN Loss: 4.378981590270996 | BCE Loss: 1.0681264400482178
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 5.400277614593506 | KNN Loss: 4.39302396774292 | BCE Loss: 1.0072537660598755
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 5.373232841491699 | KNN Loss: 4.364997863769531 | BCE Loss: 1.0082347393035889
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 5.353822708129883 | KNN Loss: 4.343193054199219 |

Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 5.379047393798828 | KNN Loss: 4.343362808227539 | BCE Loss: 1.0356848239898682
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 5.382643699645996 | KNN Loss: 4.343164920806885 | BCE Loss: 1.0394785404205322
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 5.374365329742432 | KNN Loss: 4.361779689788818 | BCE Loss: 1.0125857591629028
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 5.399739742279053 | KNN Loss: 4.361929416656494 | BCE Loss: 1.037810206413269
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 5.390772819519043 | KNN Loss: 4.3784918785095215 | BCE Loss: 1.012281060218811
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 5.411612510681152 | KNN Loss: 4.391783714294434 | BCE Loss: 1.0198289155960083
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 5.435630798339844 | KNN Loss: 4.365993499755859 | BCE Loss: 1.0696371793746948
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 5.437350273132324 | KNN Loss: 4.367086887359619 | 

Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 5.416067123413086 | KNN Loss: 4.398866176605225 | BCE Loss: 1.0172009468078613
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 5.411890029907227 | KNN Loss: 4.358218193054199 | BCE Loss: 1.0536715984344482
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 5.356133460998535 | KNN Loss: 4.349209785461426 | BCE Loss: 1.0069239139556885
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 5.366866111755371 | KNN Loss: 4.337185859680176 | BCE Loss: 1.0296804904937744
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 5.434194564819336 | KNN Loss: 4.369448184967041 | BCE Loss: 1.0647461414337158
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 5.406996250152588 | KNN Loss: 4.394435405731201 | BCE Loss: 1.0125607252120972
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 5.428384780883789 | KNN Loss: 4.412336826324463 | BCE Loss: 1.0160481929779053
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 5.390324592590332 | KNN Loss: 4.360025405883789 | 

Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 5.432872772216797 | KNN Loss: 4.369320869445801 | BCE Loss: 1.0635517835617065
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 5.410745143890381 | KNN Loss: 4.365313529968262 | BCE Loss: 1.0454316139221191
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 5.4033637046813965 | KNN Loss: 4.357863426208496 | BCE Loss: 1.0455002784729004
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 5.490099906921387 | KNN Loss: 4.437799453735352 | BCE Loss: 1.052300214767456
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 5.450606346130371 | KNN Loss: 4.41609525680542 | BCE Loss: 1.0345113277435303
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 5.399035930633545 | KNN Loss: 4.3543572425842285 | BCE Loss: 1.044678807258606
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 5.3918023109436035 | KNN Loss: 4.384982585906982 | BCE Loss: 1.006819725036621
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 5.442098617553711 | KNN Loss: 4.405357360839844 | B

In [13]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.4010e+00,  2.2324e+00,  1.9259e+00,  4.4305e+00,  4.3083e+00,
          8.2646e-01,  2.8776e+00,  2.2525e+00,  1.6196e+00,  2.5330e+00,
          2.7708e+00,  2.3513e+00,  8.7957e-01,  1.9581e+00,  1.4848e+00,
          1.2184e+00,  3.0277e+00,  1.7433e+00,  2.0688e+00,  1.9962e+00,
          1.8550e+00,  1.6882e+00,  2.8604e+00,  3.1876e+00,  1.7777e+00,
          1.9339e+00,  1.3438e+00,  9.6325e-01,  1.2171e+00,  3.6746e-01,
          7.2058e-02,  8.8119e-01,  2.8268e-01,  1.0370e+00,  1.0608e+00,
          1.9995e+00,  1.0214e+00,  4.1382e+00,  4.7325e-01,  1.4965e+00,
          1.0061e+00, -6.7459e-01, -6.6089e-01,  2.7963e+00,  1.9189e+00,
          3.0279e-01, -2.4250e-02,  2.4552e-02,  1.9926e+00,  1.7274e+00,
          1.0519e+00, -2.2927e-02,  1.2995e+00,  7.2269e-01, -4.8415e-01,
          1.5939e+00,  1.9762e+00,  1.1411e+00,  1.4182e+00,  1.0440e+00,
          4.4690e-01,  9.6211e-01,  2.4799e-01,  1.5313e+00,  1.4831e+00,
          1.7758e+00, -1.8357e+00,  5.

In [14]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [18]:
dataset_ = [d[0].cpu() for d in dataset]

In [19]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 76.28it/s]


In [20]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [21]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [22]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [24]:
tensor_dataset = torch.stack(dataset_)

In [25]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [26]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [27]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [28]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [29]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [30]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [31]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [32]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [33]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [34]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [35]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [36]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [37]:
losses = []
accs = []
sparsity = []

In [38]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 029 | Total loss: 9.632 | Reg loss: 0.009 | Tree loss: 9.632 | Accuracy: 0.000000 | 0.279 sec/iter
Epoch: 00 | Batch: 001 / 029 | Total loss: 9.619 | Reg loss: 0.009 | Tree loss: 9.619 | Accuracy: 0.000000 | 0.254 sec/iter
Epoch: 00 | Batch: 002 / 029 | Total loss: 9.609 | Reg loss: 0.008 | Tree loss: 9.609 | Accuracy: 0.000000 | 0.247 sec/iter
Epoch: 00 | Batch: 003 / 029 | Total loss: 9.603 | Reg loss: 0.008 | Tree loss: 9.603 | Accuracy: 0.000000 | 0.243 sec/iter
Epoch: 00 | Batch: 004 / 029 | Total loss: 9.588 | Reg loss: 0.008 | Tree loss: 9.588 | Accuracy: 0.000000 | 0.243 sec/iter
Epoch: 00 | Batch: 005 / 029 | Total loss: 9.586 | Reg loss: 0.008 | Tree loss: 9.586 | Accuracy: 0.000000 | 0.242 sec/iter
Epoch: 00 | Batch: 006 / 029 | Total loss: 9.570 | Reg loss: 0.008 | Tree loss: 9.570 | Accuracy: 0.003906 | 0.241 sec/iter
Epoch: 00 | Batch

Epoch: 02 | Batch: 004 / 029 | Total loss: 9.314 | Reg loss: 0.007 | Tree loss: 9.314 | Accuracy: 0.275391 | 0.241 sec/iter
Epoch: 02 | Batch: 005 / 029 | Total loss: 9.293 | Reg loss: 0.007 | Tree loss: 9.293 | Accuracy: 0.302734 | 0.241 sec/iter
Epoch: 02 | Batch: 006 / 029 | Total loss: 9.302 | Reg loss: 0.007 | Tree loss: 9.302 | Accuracy: 0.253906 | 0.24 sec/iter
Epoch: 02 | Batch: 007 / 029 | Total loss: 9.295 | Reg loss: 0.008 | Tree loss: 9.295 | Accuracy: 0.257812 | 0.24 sec/iter
Epoch: 02 | Batch: 008 / 029 | Total loss: 9.274 | Reg loss: 0.008 | Tree loss: 9.274 | Accuracy: 0.292969 | 0.24 sec/iter
Epoch: 02 | Batch: 009 / 029 | Total loss: 9.279 | Reg loss: 0.008 | Tree loss: 9.279 | Accuracy: 0.250000 | 0.24 sec/iter
Epoch: 02 | Batch: 010 / 029 | Total loss: 9.252 | Reg loss: 0.009 | Tree loss: 9.252 | Accuracy: 0.269531 | 0.24 sec/iter
Epoch: 02 | Batch: 011 / 029 | Total loss: 9.251 | Reg loss: 0.009 | Tree loss: 9.251 | Accuracy: 0.259766 | 0.24 sec/iter
Epoch: 02 | Ba

Epoch: 04 | Batch: 009 / 029 | Total loss: 8.926 | Reg loss: 0.013 | Tree loss: 8.926 | Accuracy: 0.306641 | 0.241 sec/iter
Epoch: 04 | Batch: 010 / 029 | Total loss: 8.932 | Reg loss: 0.014 | Tree loss: 8.932 | Accuracy: 0.271484 | 0.241 sec/iter
Epoch: 04 | Batch: 011 / 029 | Total loss: 8.923 | Reg loss: 0.014 | Tree loss: 8.923 | Accuracy: 0.273438 | 0.241 sec/iter
Epoch: 04 | Batch: 012 / 029 | Total loss: 8.900 | Reg loss: 0.014 | Tree loss: 8.900 | Accuracy: 0.281250 | 0.24 sec/iter
Epoch: 04 | Batch: 013 / 029 | Total loss: 8.894 | Reg loss: 0.015 | Tree loss: 8.894 | Accuracy: 0.279297 | 0.241 sec/iter
Epoch: 04 | Batch: 014 / 029 | Total loss: 8.882 | Reg loss: 0.015 | Tree loss: 8.882 | Accuracy: 0.269531 | 0.241 sec/iter
Epoch: 04 | Batch: 015 / 029 | Total loss: 8.828 | Reg loss: 0.016 | Tree loss: 8.828 | Accuracy: 0.316406 | 0.241 sec/iter
Epoch: 04 | Batch: 016 / 029 | Total loss: 8.831 | Reg loss: 0.016 | Tree loss: 8.831 | Accuracy: 0.289062 | 0.241 sec/iter
Epoch: 04

Epoch: 06 | Batch: 014 / 029 | Total loss: 8.419 | Reg loss: 0.019 | Tree loss: 8.419 | Accuracy: 0.320312 | 0.242 sec/iter
Epoch: 06 | Batch: 015 / 029 | Total loss: 8.414 | Reg loss: 0.019 | Tree loss: 8.414 | Accuracy: 0.312500 | 0.242 sec/iter
Epoch: 06 | Batch: 016 / 029 | Total loss: 8.423 | Reg loss: 0.020 | Tree loss: 8.423 | Accuracy: 0.287109 | 0.242 sec/iter
Epoch: 06 | Batch: 017 / 029 | Total loss: 8.404 | Reg loss: 0.020 | Tree loss: 8.404 | Accuracy: 0.261719 | 0.242 sec/iter
Epoch: 06 | Batch: 018 / 029 | Total loss: 8.401 | Reg loss: 0.020 | Tree loss: 8.401 | Accuracy: 0.232422 | 0.242 sec/iter
Epoch: 06 | Batch: 019 / 029 | Total loss: 8.357 | Reg loss: 0.021 | Tree loss: 8.357 | Accuracy: 0.267578 | 0.242 sec/iter
Epoch: 06 | Batch: 020 / 029 | Total loss: 8.367 | Reg loss: 0.021 | Tree loss: 8.367 | Accuracy: 0.248047 | 0.242 sec/iter
Epoch: 06 | Batch: 021 / 029 | Total loss: 8.320 | Reg loss: 0.022 | Tree loss: 8.320 | Accuracy: 0.285156 | 0.242 sec/iter
Epoch: 0

Epoch: 08 | Batch: 019 / 029 | Total loss: 7.875 | Reg loss: 0.023 | Tree loss: 7.875 | Accuracy: 0.277344 | 0.243 sec/iter
Epoch: 08 | Batch: 020 / 029 | Total loss: 7.883 | Reg loss: 0.024 | Tree loss: 7.883 | Accuracy: 0.250000 | 0.243 sec/iter
Epoch: 08 | Batch: 021 / 029 | Total loss: 7.838 | Reg loss: 0.024 | Tree loss: 7.838 | Accuracy: 0.251953 | 0.243 sec/iter
Epoch: 08 | Batch: 022 / 029 | Total loss: 7.825 | Reg loss: 0.024 | Tree loss: 7.825 | Accuracy: 0.263672 | 0.243 sec/iter
Epoch: 08 | Batch: 023 / 029 | Total loss: 7.785 | Reg loss: 0.025 | Tree loss: 7.785 | Accuracy: 0.292969 | 0.242 sec/iter
Epoch: 08 | Batch: 024 / 029 | Total loss: 7.783 | Reg loss: 0.025 | Tree loss: 7.783 | Accuracy: 0.257812 | 0.242 sec/iter
Epoch: 08 | Batch: 025 / 029 | Total loss: 7.721 | Reg loss: 0.025 | Tree loss: 7.721 | Accuracy: 0.292969 | 0.242 sec/iter
Epoch: 08 | Batch: 026 / 029 | Total loss: 7.718 | Reg loss: 0.025 | Tree loss: 7.718 | Accuracy: 0.296875 | 0.242 sec/iter
Epoch: 0

...

Epoch: 91 | Batch: 000 / 029 | Total loss: 2.023 | Reg loss: 0.024 | Tree loss: 2.023 | Accuracy: 0.281250 | 0.253 sec/iter
Epoch: 91 | Batch: 001 / 029 | Total loss: 2.015 | Reg loss: 0.024 | Tree loss: 2.015 | Accuracy: 0.273438 | 0.253 sec/iter
Epoch: 91 | Batch: 002 / 029 | Total loss: 1.990 | Reg loss: 0.024 | Tree loss: 1.990 | Accuracy: 0.277344 | 0.253 sec/iter
Epoch: 91 | Batch: 003 / 029 | Total loss: 2.008 | Reg loss: 0.024 | Tree loss: 2.008 | Accuracy: 0.296875 | 0.253 sec/iter
Epoch: 91 | Batch: 004 / 029 | Total loss: 2.060 | Reg loss: 0.024 | Tree loss: 2.060 | Accuracy: 0.250000 | 0.253 sec/iter
Epoch: 91 | Batch: 005 / 029 | Total loss: 2.062 | Reg loss: 0.024 | Tree loss: 2.062 | Accuracy: 0.287109 | 0.253 sec/iter
Epoch: 91 | Batch: 006 / 029 | Total loss: 2.055 | Reg loss: 0.024 | Tree loss: 2.055 | Accuracy: 0.291016 | 0.253 sec/iter
Epoch: 91 | Batch: 007 / 029 | Total loss: 2.026 | Reg loss: 0.024 | Tree loss: 2.026 | Accuracy: 0.283203 | 0.253 sec/iter
Epoch: 9

Epoch: 93 | Batch: 005 / 029 | Total loss: 2.065 | Reg loss: 0.024 | Tree loss: 2.065 | Accuracy: 0.275391 | 0.253 sec/iter
Epoch: 93 | Batch: 006 / 029 | Total loss: 2.024 | Reg loss: 0.024 | Tree loss: 2.024 | Accuracy: 0.273438 | 0.253 sec/iter
Epoch: 93 | Batch: 007 / 029 | Total loss: 2.014 | Reg loss: 0.024 | Tree loss: 2.014 | Accuracy: 0.285156 | 0.253 sec/iter
Epoch: 93 | Batch: 008 / 029 | Total loss: 2.005 | Reg loss: 0.024 | Tree loss: 2.005 | Accuracy: 0.271484 | 0.253 sec/iter
Epoch: 93 | Batch: 009 / 029 | Total loss: 1.999 | Reg loss: 0.024 | Tree loss: 1.999 | Accuracy: 0.296875 | 0.253 sec/iter
Epoch: 93 | Batch: 010 / 029 | Total loss: 2.023 | Reg loss: 0.024 | Tree loss: 2.023 | Accuracy: 0.302734 | 0.253 sec/iter
Epoch: 93 | Batch: 011 / 029 | Total loss: 2.045 | Reg loss: 0.024 | Tree loss: 2.045 | Accuracy: 0.285156 | 0.253 sec/iter
Epoch: 93 | Batch: 012 / 029 | Total loss: 2.034 | Reg loss: 0.024 | Tree loss: 2.034 | Accuracy: 0.281250 | 0.253 sec/iter
Epoch: 9

Epoch: 95 | Batch: 010 / 029 | Total loss: 2.074 | Reg loss: 0.023 | Tree loss: 2.074 | Accuracy: 0.242188 | 0.253 sec/iter
Epoch: 95 | Batch: 011 / 029 | Total loss: 2.027 | Reg loss: 0.023 | Tree loss: 2.027 | Accuracy: 0.281250 | 0.253 sec/iter
Epoch: 95 | Batch: 012 / 029 | Total loss: 2.002 | Reg loss: 0.023 | Tree loss: 2.002 | Accuracy: 0.285156 | 0.253 sec/iter
Epoch: 95 | Batch: 013 / 029 | Total loss: 2.040 | Reg loss: 0.023 | Tree loss: 2.040 | Accuracy: 0.279297 | 0.253 sec/iter
Epoch: 95 | Batch: 014 / 029 | Total loss: 2.048 | Reg loss: 0.023 | Tree loss: 2.048 | Accuracy: 0.263672 | 0.253 sec/iter
Epoch: 95 | Batch: 015 / 029 | Total loss: 2.046 | Reg loss: 0.023 | Tree loss: 2.046 | Accuracy: 0.291016 | 0.253 sec/iter
Epoch: 95 | Batch: 016 / 029 | Total loss: 2.046 | Reg loss: 0.023 | Tree loss: 2.046 | Accuracy: 0.261719 | 0.253 sec/iter
Epoch: 95 | Batch: 017 / 029 | Total loss: 2.031 | Reg loss: 0.023 | Tree loss: 2.031 | Accuracy: 0.281250 | 0.253 sec/iter
Epoch: 9

Epoch: 97 | Batch: 015 / 029 | Total loss: 2.021 | Reg loss: 0.023 | Tree loss: 2.021 | Accuracy: 0.265625 | 0.253 sec/iter
Epoch: 97 | Batch: 016 / 029 | Total loss: 2.035 | Reg loss: 0.023 | Tree loss: 2.035 | Accuracy: 0.267578 | 0.253 sec/iter
Epoch: 97 | Batch: 017 / 029 | Total loss: 2.035 | Reg loss: 0.023 | Tree loss: 2.035 | Accuracy: 0.285156 | 0.252 sec/iter
Epoch: 97 | Batch: 018 / 029 | Total loss: 2.013 | Reg loss: 0.023 | Tree loss: 2.013 | Accuracy: 0.277344 | 0.252 sec/iter
Epoch: 97 | Batch: 019 / 029 | Total loss: 2.018 | Reg loss: 0.023 | Tree loss: 2.018 | Accuracy: 0.248047 | 0.252 sec/iter
Epoch: 97 | Batch: 020 / 029 | Total loss: 2.018 | Reg loss: 0.023 | Tree loss: 2.018 | Accuracy: 0.281250 | 0.252 sec/iter
Epoch: 97 | Batch: 021 / 029 | Total loss: 2.021 | Reg loss: 0.023 | Tree loss: 2.021 | Accuracy: 0.271484 | 0.252 sec/iter
Epoch: 97 | Batch: 022 / 029 | Total loss: 2.047 | Reg loss: 0.023 | Tree loss: 2.047 | Accuracy: 0.275391 | 0.252 sec/iter
Epoch: 9

Epoch: 99 | Batch: 020 / 029 | Total loss: 1.987 | Reg loss: 0.023 | Tree loss: 1.987 | Accuracy: 0.312500 | 0.252 sec/iter
Epoch: 99 | Batch: 021 / 029 | Total loss: 2.042 | Reg loss: 0.023 | Tree loss: 2.042 | Accuracy: 0.275391 | 0.252 sec/iter
Epoch: 99 | Batch: 022 / 029 | Total loss: 2.025 | Reg loss: 0.023 | Tree loss: 2.025 | Accuracy: 0.269531 | 0.252 sec/iter
Epoch: 99 | Batch: 023 / 029 | Total loss: 2.006 | Reg loss: 0.023 | Tree loss: 2.006 | Accuracy: 0.292969 | 0.252 sec/iter
Epoch: 99 | Batch: 024 / 029 | Total loss: 2.000 | Reg loss: 0.023 | Tree loss: 2.000 | Accuracy: 0.291016 | 0.252 sec/iter
Epoch: 99 | Batch: 025 / 029 | Total loss: 1.985 | Reg loss: 0.023 | Tree loss: 1.985 | Accuracy: 0.312500 | 0.252 sec/iter
Epoch: 99 | Batch: 026 / 029 | Total loss: 2.071 | Reg loss: 0.023 | Tree loss: 2.071 | Accuracy: 0.267578 | 0.252 sec/iter
Epoch: 99 | Batch: 027 / 029 | Total loss: 2.016 | Reg loss: 0.023 | Tree loss: 2.016 | Accuracy: 0.271484 | 0.252 sec/iter
Epoch: 9

In [39]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [41]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 7.426229508196721


# Extract Rules

# Accumulate samples in the leaves

In [42]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 122


In [43]:
method = 'greedy'

In [44]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [45]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============


/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25 ==============
============== Pattern 2